<a href="https://colab.research.google.com/github/lbk209/topic_modeling/blob/main/tm_develop_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!pip install dash dash_bootstrap_components

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

iris = px.data.iris()
gapminder = px.data.gapminder()
tips = px.data.tips()
carshare = px.data.carshare()

figure_templates = [
    "plotly",
    "ggplot2",
    "seaborn",
    "simple_white",
    "plotly_white",
    "plotly_dark",
    "presentation",
    "xgridoff",
    "ygridoff",
    "gridon",
    "none",
]

change_figure_template = html.Div(
    [
        html.Div("Change Figure Template"),
        dcc.Dropdown(figure_templates, figure_templates[0], id="template"),
    ],
    className="pb-4",
)


app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(change_figure_template, lg=6)),
        dbc.Row(dbc.Col(html.Div(id="graphs"))),
    ],
    className="dbc p-4",
    fluid=True,
)


@app.callback(
    Output("graphs", "children"),
    Input("template", "value"),
)
def update_graph_theme(template):
    graph1 = dcc.Graph(
        figure=px.scatter(
            iris,
            x="sepal_width",
            y="sepal_length",
            color="species",
            title=f"Iris <br>{template} figure template",
            template=template,
        ),
        className="border",
    )
    graph2 = dcc.Graph(
        figure=px.scatter(
            gapminder,
            x="gdpPercap",
            y="lifeExp",
            size="pop",
            color="continent",
            hover_name="country",
            animation_frame="year",
            animation_group="country",
            log_x=True,
            size_max=60,
            title=f"Gapminder <br>{template} figure template",
            template=template,
        ),
        className="border",
    )
    graph3 = dcc.Graph(
        figure=px.violin(
            tips,
            y="tip",
            x="smoker",
            color="sex",
            box=True,
            points="all",
            hover_data=tips.columns,
            title=f"Tips <br>{template} figure template",
            template=template,
        ),
        className="border",
    )
    graph4 = dcc.Graph(
        figure=px.scatter_mapbox(
            carshare,
            lat="centroid_lat",
            lon="centroid_lon",
            color="peak_hour",
            size="car_hours",
            size_max=15,
            zoom=10,
            mapbox_style="carto-positron",
            title=f"Carshare <br> {template} figure template",
            template=template,
        ),
        className="border",
    )

    return [
        dbc.Row([dbc.Col(graph1, lg=6), dbc.Col(graph2, lg=6)]),
        dbc.Row([dbc.Col(graph3, lg=6), dbc.Col(graph4, lg=6)], className="mt-4"),
    ]


if __name__ == "__main__":
    app.run_server(debug=True)

In [44]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

# Incorporate data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = dbc.Container([
    dbc.Row([
        html.Div('My First App with Data, Graph, and Controls', className="text-primary text-center fs-3")
    ]),

    dbc.Row([
        dbc.RadioItems(options=[{"label": x, "value": x} for x in ['pop', 'lifeExp', 'gdpPercap']],
                       value='lifeExp',
                       inline=True,
                       id='radio-buttons-final')
    ]),

    dbc.Row([
        dbc.Col([
            dash_table.DataTable(data=df.to_dict('records'), page_size=12, style_table={'overflowX': 'auto'})
        ], width=6),

        dbc.Col([
            dcc.Graph(figure={}, id='my-first-graph-final')
        ], width=6),
    ]),

], fluid=True)

# Add controls to build the interaction
@callback(
    Output(component_id='my-first-graph-final', component_property='figure'),
    Input(component_id='radio-buttons-final', component_property='value')
)
def update_graph(col_chosen):
    fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

# Design

In [21]:
%%capture
figs_zip = 'moscato_distr'
!unzip -u {figs_zip}.zip

In [22]:
import os

figs_path, figs_file = figs_zip.split('_')

figs_files = [x for x in os.listdir(figs_path) if x.startswith(figs_file)]
figs_files = sorted(figs_files)

if len(figs_files) == 0:
    print('ERROR!: No fig to read')

In [ ]:
pattern = r'\d+(?=\.json)'
tids = [int(re.findall(pattern, x)[0]) for x in files]
tids

In [31]:
options = [{'label':f'Topic {t}', 'value': f} for t, f in zip(tids, figs_files)]

In [45]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px
import plotly.io as pio
import dash_bootstrap_components as dbc


# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)


# App layout
app.layout = dbc.Container([
    dbc.Row(
        html.Div('Topic Distribution', className="text-primary text-center fs-3")
    ),

    dbc.Row(
        #dcc.Dropdown(options=options, value=options[0]['value'], id="topics"),
        dbc.Col(
            dcc.Dropdown(options=options, value=options[0]['value'], id="topics"),
            #width="auto"
            width=4,
            ),
        justify="center"
    ),

    dbc.Row(
        dcc.Graph(figure={}, id='topic_distribution')
    ),

], fluid=True)


# Add controls to build the interaction
@callback(
    Output(component_id='topic_distribution', component_property='figure'),
    Input(component_id='topics', component_property='value')
)
def plot_topic_distr(file):
    f = f'{figs_path}/{file}'
    fig = pio.read_json(f)
    return fig


# Run the app
if __name__ == '__main__':
    app.run(debug=False,
            jupyter_width=800 #"70%"
            )


<IPython.core.display.Javascript object>